In [ ]:
#Run this command when the file is opened 
! pip install streamlit -q
!pip install pyngrok
!pip install easyocr
!pip install Pillow
!pip install opencv-python

In [165]:
%%writefile app.py

import easyocr as ocr
import streamlit as st
import PIL
from PIL import Image
from PIL import ImageDraw
import numpy as np
import sqlite3
from io import BytesIO

#====================================(title)=================================================
st.set_page_config(layout='wide')

st.title("Easy OCR - Extract Text from Images")
st.image("https://global-uploads.webflow.com/636bdbebfc681f083e923f81/63861e96a17fc270749839bc_6296ed3f6519c0e160b16515_OCR%2520%2520Main%2520Image-p-1600.png", width = 300)




#------------------------------------------(OCR Model)------------------------------------------
@st.cache_data(show_spinner= "Loading Model")
def ocr_model(): 
    reader = ocr.Reader(['en'],model_storage_directory='.', gpu = False) 
    return reader 

#@st.cache_data(show_spinner= "Extracting Text")
def draw_boxes(image,bounds,color='red',width=2):
  draw=ImageDraw.Draw(image)
  for bound in bounds:
    p0,p1,p2,p3=bound[0]
    draw.line([*p0,*p1,*p2,*p3,*p0],fill=color,width=width)
  return image



#------------------------------------------(SQL Query)------------------------------------------
@st.cache_data
def insertBLOB(filename,im_data,extracted_text):
    try:
        sqliteConnection = sqlite3.connect("bizcardx_db.db")
        cursor = sqliteConnection.cursor()

        create_table=f"""CREATE TABLE IF NOT EXISTS {filename} (
                    "filename"	TEXT NOT NULL,
                    "image"	BLOB NOT NULL,
                    "extracted_text"	TEXT NOT NULL,
                    PRIMARY KEY("filename"))"""
        cursor.execute(create_table)

        insert_query = f""" INSERT INTO {filename} (filename, image, extracted_text) VALUES (?, ?, ?)"""  
        data_tuple = (filename,im_data,extracted_text)
        cursor.execute(insert_query,data_tuple)

        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        st.error("Failed to insert blob data into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()


def readBlobData(saved_name):
  try:
      sqliteConnection = sqlite3.connect("bizcardx_db.db")
      cursor = sqliteConnection.cursor()
      #st.write("Connected to SQLite")

      view_query = f"SELECT * from {saved_name}"
      cursor.execute(view_query)
      record = cursor.fetchall()

      for row in record:
          name = row[0]
          photo = row[1]
          text = row[2]
          
      cursor.close()
      return name, photo, text

  except :
      st.error("Details Not found")
  finally:
      if sqliteConnection:
          sqliteConnection.close()


def del_details(file_name):
    try:
        sqliteConnection = sqlite3.connect("bizcardx_db.db")
        cursor = sqliteConnection.cursor()
        #st.write("Connected to SQLite")
        
        drop_query = f"DROP TABLE {file_name} "
        cursor.execute(drop_query)
        
        sqliteConnection.commit()
        cursor.close()
    except:
        st.write("Failed to delete data from sqlite table")
    finally:
        if sqliteConnection:
            sqliteConnection.close()



#=====================================(Web Page)==========================================


st.markdown("## Optical Character Recognition - Using `easyocr`, `streamlit`")

menu = ["Select Option","Upload Image", "View Details", "Delete Details"]
option = st.selectbox("options",menu)



#---------------------------------------("Upload Image")--------------------------------
if option == "Upload Image":

    #image uploader
    image = st.file_uploader(label = "Upload your image here",type=['png','jpg','jpeg'])
    reader = ocr_model()

    if image is not None:
        bytes_data = image.getvalue()
        col1, col2 = st.columns(2)

        with col1:       
        
          input_image=PIL.Image.open(image)

          with st.spinner("Extracting text..."):
            
            #reading text from image
            result=reader.readtext(np.array(input_image),detail=1,paragraph=True)
            
            #making list of the extracted texts
            result_text=[]
 
            for text in result:
                result_text.append(text[1])


            #drawing boxes in image over the detected texts
            st.write(' Detected texts')
            st.image(draw_boxes(input_image,result))
            

            
        with col2:
          with st.form("my_form"):           
            st.write('Edit the extracted text before saving')
            edited_list = st.experimental_data_editor(result_text,num_rows="dynamic")
            filename=st.text_input('Save file as')
            st.caption('-->Save filename with Contact no.as it creates unique value')
            submitted = st.form_submit_button("Save to DB")

        if submitted and filename:
            extracted_text=''
            for text in edited_list:
                extracted_text+=' '+text

            insertBLOB(filename,bytes_data,extracted_text)
            st.snow()

    st.markdown('''Made with ❤️ by [Prashanth M](https://github.com/prashanth-githubuser)''')




#---------------------------------------("View Details")--------------------------------
if option == "View Details":
  
    filename=st.text_input('Enter the Contact No.')
    submitted = st.button("View Details")

    if filename and submitted:
      try:
        name, photo, text= readBlobData(filename)
        col1, col2 = st.columns(2)

        with col1:
        #st.write(name)
          st.subheader("Business Card")
          image_data = BytesIO(photo)
          img = Image.open(image_data)
          st.image(img)

        with col2:
          st.subheader("Details")
          st.write(text)
      except:
        st.info("Please Upload the Business Card")


    st.markdown('''Made with ❤️ by [Prashanth M](https://github.com/prashanth-githubuser)''')





#---------------------------------------("Delete Details")--------------------------------
if option == "Delete Details":

    file_name=st.text_input('Enter the file name', key='verification_1')

    submitted_button_1 = st.button("Delete Details", key = 'key1')
    st.error('Warning Once deleted, data cannot be recovered', icon= "🚨")

    if file_name and submitted_button_1: 
          try:
              sqliteConnection = sqlite3.connect("bizcardx_db.db")
              cursor = sqliteConnection.cursor()
              #st.write("Connected to SQLite")
              
              drop_query = f"DROP TABLE {file_name}"
              cursor.execute(drop_query)
              st.success("Details Deleted successfully")
              
              sqliteConnection.commit()
              cursor.close()
          except:
              st.error("File not found")
          finally:
              if sqliteConnection:
                  sqliteConnection.close()   
          
    elif submitted_button_1:
      st.info("Please check the file name")

    st.markdown('''Made with ❤️ by [Prashanth M](https://github.com/prashanth-githubuser)''')


            



Overwriting app.py


In [166]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.27.47:8501

npx: installed 22 in 3.45s
your url is: https://breezy-dingos-tickle-35-229-27-47.loca.lt
2023-04-15 05:50:32.810 Using CPU. Note: This module is much faster with a GPU.
/usr/local/lib/python3.9/dist-packages/torch/_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,
  Stopping...
^C
